<h1>Step 1. Importing and cleaning the data </h1>

In [1]:
# Import packages
import numpy as np
import pandas as pd
import os
import datetime as dt
from datetime import timedelta
import pickle

# Import for plot
import matplotlib.pyplot as plt

# Import custom functions
import bb_clean_data

In [2]:
# Set directories and other file parameters
main_dir = os.getcwd() 
data_dir = main_dir + '/data'
filename_pre = 'indego_trips_'
file_format = '.csv'
year_range = ['2015','2016','2017','2018'] 
column_names = ['trip_id', 'duration', 'start_time', 'end_time','start_station','start_lat','start_lon', 
                    'end_station','end_lat','end_lon', 'bike_id', 'plan_duration', 'trip_route_category', 'passholder_type']
which_year = '2015'
station_numbers = [3000, 4000] # REVISIT THIS TO EXCLUDE DUMMY STATIONS (very few and likely no rides)
min_duration = 1 # 1 minute
max_duration_sd = 2 # 2 standard deviations 

filenames = bb_clean_data.get_filenames_year(which_year, filename_pre, file_format)
df = bb_clean_data.create_and_clean_df(which_year, filenames, data_dir, column_names, min_duration, max_duration_sd)

# Get the data and save them for analyzing the rides over time (aggregated by month)
df['month'] = pd.DatetimeIndex(df['start_time']).month
if which_year == '2018':
    df = df[df['month'] < 10] # before october
elif which_year == '2015': 
    df = df[df['month'] > 4] # after april
rides_per_month = df.groupby('month').count()['trip_id']
filename = 'rides_per_month'+ str(which_year) + '.pckl'
f = open(filename, 'wb')
pickle.dump(rides_per_month, f)
f.close()

# Calculate % passholders of different types across the network for this year
passholders = False 
if passholders:
    passholder_type_agg = df.groupby('passholder_type').count()['trip_id']/sum(df.groupby('passholder_type').count()['trip_id'])
    passholder_labels = passholder_type_agg.index.values.tolist()
    print(passholder_type_agg)
    fig1, ax1 = plt.subplots()
    ax1.pie(passholder_type_agg, labels = passholder_labels, autopct ='%1.1f%%',
        shadow = True, startangle = 90)
    ax1.axis('equal')  # Note from web: Equal aspect ratio ensures that pie is drawn as a circle.
    plt.show()    

# Make a timeseries array
first_day = df['start_time_date'].min().to_timestamp()
last_day = df['start_time_date'].max().to_timestamp()  + timedelta(hours=23)
timeseries_format =  pd.date_range(first_day, last_day, freq='H')

# Identify most valuable stations
how_many = 10
station_df = df.groupby(['start_station'])['trip_id'].count() #identify unique trips and count them
most_valuable_stations = station_df.sort_values(ascending = False).head(how_many).index.tolist()
print(how_many,"most valuable stations in rank-order are: ", most_valuable_stations)

# Save the set of stations if it's for 2018 year
if which_year == 2018: 
    import pickle
    f = open('most_valuable_stations.pckl', 'wb')
    pickle.dump(most_valuable_stations, f)
    f.close()
    
# Option to look at the breakdown of passholders per station
passholders_per_station = False
for station in most_valuable_stations: 
    bb_clean_data.join_df_with_timeseries(df, timeseries_format, station, which_year, file_format)   
    if passholders_per_station:
        st = df[df['start_station_id'] == station] 
        print(station)
        st_passholder_type_agg = st.groupby('passholder_type').count()['trip_id']/sum(st.groupby('passholder_type').count()['trip_id'])
        st_passholder_type_agg.round(decimals = 2)         

/Users/ana/anaconda3/envs/insight/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


For year 2015: 
Percent of trips that are too short 1.7.
Percent of trips that are too long 1.7.
10 most valuable stations in rank-order are:  [3023, 3010, 3045, 3032, 3007, 3022, 3057, 3052, 3021, 3063]
